# Machine Learning con datos en PostgreSQL
## Andrea Exarahí y Fabián Gómez

# Insercion de datos a PostgreSQL

#### Creacion de la tabla

Por medio de este script de base de datos por medio de la terminal creamos la tabla "projects" que va a tener el set de datos elegido para este proyecto

Una vez creada la tabla por medio de este comando copiamos la data en la tabla

In [1]:
#CSV does have headers, they need to match the columns with the table
\copy projects from '/Users/caro/Documents/BigData/Proyecto/ks-projects.csv' csv header;

SyntaxError: unexpected character after line continuation character (<ipython-input-1-ed1fd751cab7>, line 2)

Set de datos escogido para este proyecto fue tomado de kaggle

https://www.kaggle.com/kemical/kickstarter-projects#ks-projects-201801.csv

Esta data contiene registros de mas de 300,000 proyectos de kickstarter, un sitio web de crowdfounding para proyectos creativos.Mediante Kickstarter se ha financiado una amplia gama de esfuerzos,que van desde películas independientes, música y cómics a periodismo, videojuegos y proyectos relacionados con la comida.

#### Problema

Seleccionamos este set de datos ya que no gusta mucho esta plataforma de crowdfounding y en algunas ocaciones hemos participado en campañas para ayudar el proyecto o adquirir de los produtos innovadores que se encuentran en la plataforma.

Tenemos curiosidad de conocer si se puede predecir cuales proyectos son exitosos y cuales no lo logran en la plataforma.

# Importacion de librerias y operaciones

In [4]:
import findspark
findspark.init('/opt/spark/spark')
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql import *
from  pyspark.sql.functions import regexp_replace,col
import re
from pyspark.sql.types import StructField,IntegerType, StructType,StringType, DateType, decimal
from pyspark.sql.functions import col, date_format, udf,quarter, lit, unix_timestamp
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler
from pyspark.ml.regression import LinearRegression
from datetime import datetime


# Sesión de Spark

Creamos la sesión de spark para trabajar con los datos en postgreSQL

In [5]:
#Spark session
spark = SparkSession \
    .builder \
    .appName("Final project") \
    .config("spark.driver.extraClassPath", "/opt/postgresql-42.2.9.jar") \
    .config("spark.executor.extraClassPath", "/opt/postgresql-42.2.9.jar") \
    .getOrCreate()

Leemos el dataframe en Spark

In [6]:
# Reading single DataFrame in Spark by retrieving all rows from a DB table.
df = spark \
    .read \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://localhost/ks_projects") \
    .option("user", "caro") \
    .option("password", "Caro2017") \
    .option("driver", "org.postgresql.Driver") \
    .option("dbtable", "projects") \
    .load()

df.head(2)

[Row(id=1000002330, name='The Songs of Adelaide & Abullah', category='Poetry', main_category='Publishing', currency='GBP', deadline='10/9/15', goal='1000', launched='8/11/15 12:12', pledged='0', state='failed', backers='0', country='GB', usd_pledged=Decimal('0.00'), usd_pledged_real=Decimal('0.00'), usd_goal_real=Decimal('1533.95')),
 Row(id=1000003930, name='Greeting From Earth: ZGAC Arts Capsule For ET', category='Narrative Film', main_category='Film & Video', currency='USD', deadline='11/1/17', goal='30000', launched='9/2/17 4:43', pledged='2421', state='failed', backers='15', country='US', usd_pledged=Decimal('100.00'), usd_pledged_real=Decimal('2421.00'), usd_goal_real=Decimal('30000.00'))]

# Limpieza de los datos

Primero veamos el schema de los datos.

In [7]:
df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- category: string (nullable = true)
 |-- main_category: string (nullable = true)
 |-- currency: string (nullable = true)
 |-- deadline: string (nullable = true)
 |-- goal: string (nullable = true)
 |-- launched: string (nullable = true)
 |-- pledged: string (nullable = true)
 |-- state: string (nullable = true)
 |-- backers: string (nullable = true)
 |-- country: string (nullable = true)
 |-- usd_pledged: decimal(38,2) (nullable = true)
 |-- usd_pledged_real: decimal(38,2) (nullable = true)
 |-- usd_goal_real: decimal(38,2) (nullable = true)



|     Columna   | Descripcion  |
| ------------- |:-------------:|
| id     | internal kickstarter id|
|name    | name of project - A project is a finite work with a clear goal that you’d like to bring to life. Think albums, books, or films.|
|category| category   |
|main_category| category of campaign|
|currency| currency used to support    |
|deadline| deadline for crowdfunding |
|goal| fundraising goal - The funding goal is the amount of money that a creator needs to complete their project. |
|launched | date launched|
|pledged| amount pledged by "crowd"      |
|state|Current condition the project is in| 
|backers|number of backers|
|country |country pledged from|
|usd_pledged| Pledged amount in USD |
|usd_pledged_real     | Pledged amount in USD (conversion made by fixer.io api|
|usd_goal_real  |Goal amount in USD|

Conociendo la data vamos a empezar con la limpieza y escogencia de variables a estudiar

Empezaremos con la cantidad de categorias:

In [8]:
df.select(F.col('category')).distinct().count()

159

Existe 159 diferentes categorias, por lo cual tal vez descartemos esta columna como hiperparametro, ya que tambien existe una columna que contiene la categoria principal.

Categorias principales:

In [9]:
df.select(F.col('main_category')).distinct().show()

+-------------+
|main_category|
+-------------+
|         Food|
|          Art|
|      Fashion|
| Film & Video|
|   Publishing|
|       Crafts|
|       Comics|
|        Games|
|        Music|
|       Design|
|  Photography|
|   Technology|
|   Journalism|
|      Theater|
|        Dance|
+-------------+



Existen 15 distintas categorias principales para los proyectos.

Ya que este puede ser un parametro significativo para estudio, vamos a limpiar la columna sustituyendo los strings por ids numericos de la siguiente manera:

|     ID   | Categoria  |
| ------------- |:-------------:|
| 1| Art|
| 2| Fashion|
| 3| Film & Video|
| 4| Publishing|
| 5| Crafts|
| 6| Comics|
| 7| Games|
| 8| Music|
| 9| Design|
|10| Photography| 
|11| Technology|
|12| Journalism|
|13| Theater|
|14| Dance|
|15| Food|

Por medio de regexp_replace remplazamos los nombres de las categorias por los IDs

In [10]:
df = df.withColumn('main_category', regexp_replace('main_category', 'Art', '1'))
df = df.withColumn('main_category', regexp_replace('main_category', 'Fashion', '2'))
df = df.withColumn('main_category', regexp_replace('main_category', 'Film & Video', '3'))
df = df.withColumn('main_category', regexp_replace('main_category', 'Publishing','4'))
df = df.withColumn('main_category', regexp_replace('main_category', 'Crafts','5'))
df = df.withColumn('main_category', regexp_replace('main_category', 'Comics','6'))
df = df.withColumn('main_category', regexp_replace('main_category', 'Games','7'))
df = df.withColumn('main_category', regexp_replace('main_category', 'Music','8'))
df = df.withColumn('main_category', regexp_replace('main_category', 'Design','9'))
df = df.withColumn('main_category', regexp_replace('main_category', 'Photography','10'))
df = df.withColumn('main_category', regexp_replace('main_category', 'Technology','11'))
df = df.withColumn('main_category', regexp_replace('main_category', 'Journalism','12'))
df = df.withColumn('main_category', regexp_replace('main_category', 'Theater','13'))
df = df.withColumn('main_category', regexp_replace('main_category', 'Dance','14'))
df = df.withColumn('main_category', regexp_replace('main_category', 'Food','15'))

Visualizamos el cambio en la columna

In [11]:
df.select(F.col('main_category')).distinct().show()

+-------------+
|main_category|
+-------------+
|            7|
|           15|
|           11|
|            3|
|            8|
|            5|
|            6|
|            9|
|            1|
|           10|
|            4|
|           12|
|           13|
|           14|
|            2|
+-------------+



Vamos a hacer el mismo proceso para el estado de los proyectos, vayamos a ultilazar esta columna como el Y, ya que el proyecto se va a enfocar en la prediccion de proyectos que terminan success en kickstarted

In [12]:
df.select(F.col('state')).distinct().show(20)

+----------+
|     state|
+----------+
| suspended|
|    failed|
|  canceled|
|successful|
|      live|
| undefined|
+----------+



Podemos visualizar 6 diferentes estados de los proyectos, de estos vamos a tomar cuatro para estudio.

suspended, failed, y canceled van a tener un ID 0 como proyectos que no fueron exitosos y successful tendra un ID 1 de que ha sido un proyecto exitoso.

In [13]:
df = df.withColumn('state', regexp_replace('state', 'successful', '1'))
df = df.withColumn('state', regexp_replace('state', 'failed', '0'))
df = df.withColumn('state', regexp_replace('state', 'canceled', '0'))
df = df.withColumn('state', regexp_replace('state', 'suspended', '0'))

En el caso de live y undefined, lo vamos a rellenar con nulls, ya que no sabemos si realmente fueron exitosos o no.

La idea es que los estados de los proyectos queden de la siguiente manera:

|     state   | ID  |
| ------------- |:-------------:|
| successful| 1|
| failed| 0|
| canceled| 0|
| suspended| 0|
| undefined| no aplica/null|
| live| no aplica/null|

In [14]:
 df =  df.withColumn("state",F.when(col("state").isin('undefined', 'live'),None).otherwise(col("state")))

Ahora vamos a visualizar los diferentes estados y la cantidad de estos en el set de datos

In [15]:
df.groupBy('state').count().show(23)

+-----+------+
|state| count|
+-----+------+
|    0|238344|
| null|  6361|
|    1|133956|
+-----+------+



Vamos a visualizar la cantidad de distinatas monedas en las que se realizaron los proyectos:

In [16]:
df.select(F.col('currency')).distinct().show(20)

+--------+
|currency|
+--------+
|     DKK|
|     NZD|
|     GBP|
|     CHF|
|     CAD|
|     EUR|
|     NOK|
|     AUD|
|     JPY|
|     HKD|
|     MXN|
|     USD|
|     SEK|
|     SGD|
+--------+



Currency, es un campo que no tomaremos como hiperparamento debido a que la mayoria de los montos se encuentran en los datos USD y ademas contamos con otro parametro "country" o pais el cual puede ser mas significativo demograficamente.

Primero vamos a visualizar los distintos paises y la cantidad de proyectos asociados a cada uno

In [17]:
df.groupBy('country').count().show(25)

+-------+------+
|country| count|
+-------+------+
|     NL|  2868|
|     MX|  1752|
|     AT|   597|
|     HK|   618|
|     AU|  7839|
|     CA| 14756|
|     GB| 33672|
|     DE|  4171|
|     ES|  2276|
|     US|292627|
|     FR|  2939|
|     CH|   761|
|     SG|   555|
|     IT|  2878|
|     SE|  1757|
|   N,0"|  3797|
|     JP|    40|
|     NZ|  1447|
|     IE|   811|
|     BE|   617|
|     NO|   708|
|     LU|    62|
|     DK|  1113|
+-------+------+



Tal como se pudo visualizar hay algunos proyectos que no tienen un pais asociado, estos tienen una abreviacion 'N,0"', estos los vamos a sustituir con un null debido a que no es una cantidad significativa de valores.

#Contries

|ID|Abreviacion|Pais|
|-----|:-------:|-----------
|1|   NL|NETHERLANDS
|2|   MX|MEXICO
|3|   AT|AUSTRIA
|4|   HK|HONG KONG
|5|   AU|AUSTRALIA
|6|   CA|CANADA
|7|   GB|UNITED KINGDOM
|8|   DE|GERMANY
|9|   ES|SPAIN
|10|  US|UNITED STATES
|11|  FR|FRANCE
|12|  CH|SWITZERLAND
|13|  IT|ITALY
|14|  SE|SWEDEN
|null|N,0"|NO COUNTRY
|15|  JP|JAPAN
|16|  NZ|NEW ZEALAND
|17|  IE|IRELAND
|18|  BE|BELGIUM
|19|  NO|NORWAY
|20|  LU|LUXEMBOURG
|22|  DK|DENMARK
|23|  SG|SINGAPORE


In [18]:
df = df.withColumn('country', regexp_replace('country', 'NL', '1'))
df = df.withColumn('country', regexp_replace('country', 'MX', '2'))
df = df.withColumn('country', regexp_replace('country', 'AT', '3'))
df = df.withColumn('country', regexp_replace('country', 'HK', '4'))
df = df.withColumn('country', regexp_replace('country', 'AU', '5'))
df = df.withColumn('country', regexp_replace('country', 'CA', '6'))
df = df.withColumn('country', regexp_replace('country', 'GB', '7'))
df = df.withColumn('country', regexp_replace('country', 'DE', '8'))
df = df.withColumn('country', regexp_replace('country', 'ES', '9'))
df = df.withColumn('country', regexp_replace('country', 'US', '10'))
df = df.withColumn('country', regexp_replace('country', 'FR', '11'))
df = df.withColumn('country', regexp_replace('country', 'CH', '12'))
df = df.withColumn('country', regexp_replace('country', 'IT', '13'))
df = df.withColumn('country', regexp_replace('country', 'SE', '14'))
df = df.withColumn('country', regexp_replace('country', 'JP', '15'))
df = df.withColumn('country', regexp_replace('country', 'NZ', '16'))
df = df.withColumn('country', regexp_replace('country', 'IE', '17'))
df = df.withColumn('country', regexp_replace('country', 'BE', '18'))
df = df.withColumn('country', regexp_replace('country', 'NO', '19'))
df = df.withColumn('country', regexp_replace('country', 'LU', '20'))
df = df.withColumn('country', regexp_replace('country', 'DK', '21'))
df = df.withColumn('country', regexp_replace('country', 'SG', '22'))

In [19]:
df =  df.withColumn("country",F.when(col("country").isin('N,0"'),None).otherwise(col("country")))

Visualizamos el cambio en la columna

In [20]:
df.groupBy('country').count().show(23)

+-------+------+
|country| count|
+-------+------+
|      7| 33672|
|     15|    40|
|     11|  2939|
|      3|   597|
|      8|  4171|
|     22|   555|
|     16|  1447|
|   null|  3797|
|      5|  7839|
|     18|   617|
|     17|   811|
|      6| 14756|
|     19|   708|
|      9|  2276|
|      1|  2868|
|     20|    62|
|     10|292627|
|      4|   618|
|     12|   761|
|     13|  2878|
|     14|  1757|
|     21|  1113|
|      2|  1752|
+-------+------+



Ahora vamos a eliminar las columnas 'name', 'category','currency' ya que decidimos que no van a ser relevantes para nuestro problema

In [21]:
columns_to_drop = ['name', 'category','currency']
df = df.drop(*columns_to_drop)

Visualizamos las columnas que vamos a utilizar

In [22]:
df.head(1)

[Row(id=1000002330, main_category='4', deadline='10/9/15', goal='1000', launched='8/11/15 12:12', pledged='0', state='0', backers='0', country='7', usd_pledged=Decimal('0.00'), usd_pledged_real=Decimal('0.00'), usd_goal_real=Decimal('1533.95'))]

Ahora vamos a comvertir los campos que contienen fechas en datetimes.

In [23]:
string_to_date = \
    udf(lambda text_date: datetime.strptime(text_date, '%m/%d/%y %H:%M'),
        DateType())

string_to_date_deadLine = \
    udf(lambda text_date: datetime.strptime(text_date, '%m/%d/%y'),
        DateType())


df2 = df.withColumn("dateLaunched", string_to_date(df.launched))
df3 = df2.withColumn("dateDeadLine", string_to_date_deadLine(df2.deadline))


Visualizamos el resultado:

In [24]:
df3.head(1)

[Row(id=1000002330, main_category='4', deadline='10/9/15', goal='1000', launched='8/11/15 12:12', pledged='0', state='0', backers='0', country='7', usd_pledged=Decimal('0.00'), usd_pledged_real=Decimal('0.00'), usd_goal_real=Decimal('1533.95'), dateLaunched=datetime.date(2015, 8, 11), dateDeadLine=datetime.date(2015, 10, 9))]

Vamos a agregar otra columna derivada de la fecha en que inicio el proyecto, la columna la vamos a llamar "quarter" y va a tener el cuatrimestre en el que se lanzo el proyecto

In [25]:
df4 = df3.withColumn("quarter", quarter(df3.dateLaunched))

Visualizamos el resultado

In [26]:
df4.head(1)

[Row(id=1000002330, main_category='4', deadline='10/9/15', goal='1000', launched='8/11/15 12:12', pledged='0', state='0', backers='0', country='7', usd_pledged=Decimal('0.00'), usd_pledged_real=Decimal('0.00'), usd_goal_real=Decimal('1533.95'), dateLaunched=datetime.date(2015, 8, 11), dateDeadLine=datetime.date(2015, 10, 9), quarter=3)]

Vamos a borrar las columnas que no estan estandarizadas

In [27]:
#borrar columnas
columns_to_drop = ['deadline', 'launched']
df4 = df4.drop(*columns_to_drop)

Resultado:

In [28]:
df4.head(1)

[Row(id=1000002330, main_category='4', goal='1000', pledged='0', state='0', backers='0', country='7', usd_pledged=Decimal('0.00'), usd_pledged_real=Decimal('0.00'), usd_goal_real=Decimal('1533.95'), dateLaunched=datetime.date(2015, 8, 11), dateDeadLine=datetime.date(2015, 10, 9), quarter=3)]

Ahora vamos a hacer un conteo de la data que tenemos actualimente, incluyendo nulls

In [29]:
df4.count()

378661

Vamos a quitar esos nulls

In [30]:
#eliminar NaN
df4 = df4.dropna()
df4.count()

372066

# Exploracion de los datos

Para este estudio vamos a primero algunos counts para ver datos significativos

#### Proyectos exitosos por pais

In [31]:
df4.where(df4['state'] == '1').groupBy("country").count().sort("count", ascending=False).show(25)

+-------+------+
|country| count|
+-------+------+
|     10|109299|
|      7| 12067|
|      6|  4134|
|      5|  2010|
|      8|   937|
|     11|   908|
|      1|   617|
|     14|   509|
|      9|   492|
|     16|   448|
|     13|   439|
|      2|   396|
|     21|   360|
|      4|   216|
|     17|   207|
|     12|   187|
|     22|   178|
|     19|   162|
|     18|   152|
|      3|   107|
|     20|    19|
|     15|     7|
+-------+------+



Tal como podemos visualizar el país con más proyectos exitosos es Estados Unidos, con más de cienmil proyectos, seguido por Reino Unido. Por último, se encuentra Japon con solo 7 proyectos exitosos

#### Paises con más proyectos fallidos en kickstarted:

In [32]:
df4.where(df4['state'] == '0').groupBy("country").count().sort("count", ascending=False).show(25)

+-------+------+
|country| count|
+-------+------+
|     10|181588|
|      7| 21326|
|      6| 10490|
|      5|  5759|
|      8|  3159|
|     13|  2363|
|      1|  2216|
|     11|  1979|
|      9|  1732|
|      2|  1249|
|     14|  1228|
|     16|   988|
|     21|   737|
|     17|   593|
|     12|   560|
|     19|   538|
|      3|   475|
|     18|   453|
|      4|   367|
|     22|   349|
|     20|    42|
|     15|    24|
+-------+------+



Tal como podemos visualizar el país con más proyectos fallidos es Estados Unidos, con más de ciento ochentamil proyectos, seguido por Reino Unido. Por último, se encuentra Japon con solo 24 proyectos exitosos.

De acuerdo a estos datos podriamos asumir que esto se debe a la proporcion de proyectos por pais, mas que por el pais realmente.

Entonces visualicemos la cantidad de proyectos por pais, para comprobar esta hipotesis.

In [33]:
df4.groupBy('country').count().sort("count", ascending=False).show(25)

+-------+------+
|country| count|
+-------+------+
|     10|290887|
|      7| 33393|
|      6| 14624|
|      5|  7769|
|      8|  4096|
|     11|  2887|
|      1|  2833|
|     13|  2802|
|      9|  2224|
|     14|  1737|
|      2|  1645|
|     16|  1436|
|     21|  1097|
|     17|   800|
|     12|   747|
|     19|   700|
|     18|   605|
|      4|   583|
|      3|   582|
|     22|   527|
|     20|    61|
|     15|    31|
+-------+------+



A simple vista no se ve un patron entre las proporciones de proyectos exitosos y fallidos por pais, pero si se ve que la cantidad va relacionada a la cantidad de proyectos por pais

#### Proyectos exitosos por quarter

In [34]:
df4.where(df4['state'] == '1').groupBy("quarter").count().sort("count", ascending=False).show(25)

+-------+-----+
|quarter|count|
+-------+-----+
|      2|35862|
|      3|33732|
|      1|33295|
|      4|30962|
+-------+-----+



Podemos visualizar que no hay mucha relevancia en la fecha en que inician los proyectos ya que aunque hay diferencias entre los quarters en que se lanzan los proyectos, esta no es tan significativa.

#### Proyectos exitosos por categoria

In [35]:
df4.where(df4['state'] == '1').groupBy("main_category").count().sort("count", ascending=False).show(25)

+-------------+-----+
|main_category|count|
+-------------+-----+
|            8|24105|
|            3|23612|
|            7|12518|
|            4|12300|
|            1|11510|
|            9|10549|
|           13| 6534|
|           11| 6433|
|           15| 6085|
|            6| 5842|
|            2| 5593|
|           10| 3305|
|           14| 2338|
|            5| 2115|
|           12| 1012|
+-------------+-----+



Las categorias más exitosas son:

- Music
- Film & Video
- Games
- Publishing
- Art
- Desing

In [36]:
df4.groupBy('main_category').count().sort("count", ascending=False).show(25)

+-------------+-----+
|main_category|count|
+-------------+-----+
|            3|62365|
|            8|49249|
|            4|39080|
|            7|34939|
|           11|32185|
|            9|29762|
|            1|27959|
|           15|24416|
|            2|22562|
|           13|10871|
|            6|10743|
|           10|10730|
|            5| 8733|
|           12| 4723|
|           14| 3749|
+-------------+-----+



Al ver la totalidad de proyectos por categoria, si se puede asumir que hay categorias más exitosas que otras no directamente relacionado a la cantidad de proyectos, ya que por ejemplo "Technology" se encuentra en el top 5 de categorias con mas proyectos, pero en cuanto a proyectos exitosos este se encuentra el noveno lugar.

#### Proyectos exitosos de acuerdo a la cantidad que el proyecto fijo como meta "usd_goal_real"

In [113]:
df4.where(df4['state'] == '1').groupBy("usd_goal_real").count().sort("count", ascending=False).show(25)

+-------------+-----+
|usd_goal_real|count|
+-------------+-----+
|      5000.00| 8352|
|     10000.00| 6566|
|      1000.00| 5854|
|      3000.00| 5537|
|      2000.00| 5304|
|      2500.00| 4299|
|       500.00| 4205|
|      1500.00| 3809|
|     15000.00| 3750|
|      4000.00| 3107|
|     20000.00| 2893|
|      3500.00| 2534|
|      6000.00| 2484|
|     25000.00| 2092|
|      8000.00| 2057|
|      7500.00| 1630|
|     12000.00| 1566|
|      7000.00| 1546|
|     30000.00| 1506|
|     50000.00| 1356|
|      1200.00| 1236|
|       300.00| 1189|
|       600.00| 1093|
|      4500.00|  999|
|       800.00|  962|
+-------------+-----+
only showing top 25 rows



Basicamente podemos ver que la cantidad de dinero que se fija como meta no es muy relevante para que el proyecto sea exitoso o no, ya que varia mucho entre cantidades.

# Feature Engineering

Antes de iniciar con el Feature Engineering convertiremos las fechas en double(unix), ya que para hacer el vector de features es necesario contar con datos numericos.

In [167]:
df5 = df4.withColumn("unix_dateLaunched" , unix_timestamp("dateLaunched") )
df6 = df5.withColumn("unix_dateDeadLine" , unix_timestamp("dateDeadLine") )
df6.head(1)

[Row(id=1000002330, main_category='4', goal='1000', pledged='0', state='0', backers='0', country='7', usd_pledged=Decimal('0.00'), usd_pledged_real=Decimal('0.00'), usd_goal_real=Decimal('1533.95'), dateLaunched=datetime.date(2015, 8, 11), dateDeadLine=datetime.date(2015, 10, 9), quarter=3, unix_dateLaunched=1439272800, unix_dateDeadLine=1444370400)]

Ahora hacemos un cast de las columnas para que estas se puedan convertir en un vector ensamblador, vamos a omitir las siguientes columnas:

- ***id*** = not a feature
- ***dateLaunched*** = unix_dateLaunched
- ***dateDeadLine*** = unix_dateDeadLine
- ***goal*** = usd_goal_real
- ***pledged*** = usd_pledged_real
- ***usd_pledged*** = usd_pledged_real

esto debido a que son equivalentes a otras que ya estan en la data 

In [168]:
df_num = df6.select(
    df6.main_category.cast("int"),
    df6.backers.cast("float"),
    df6.country.cast("int"),
    df6.usd_pledged_real.cast("float"),
    df6.usd_goal_real.cast("float"),
    df6.unix_dateLaunched.cast("long"),
    df6.unix_dateDeadLine.cast("long"),
    df6.quarter.cast("int"),
    df6.state.cast("int").alias("label"))

Imprimimos el resultado y el esquema del mismo

In [169]:
df_num.printSchema()
df_num.head(1)

root
 |-- main_category: integer (nullable = true)
 |-- backers: float (nullable = true)
 |-- country: integer (nullable = true)
 |-- usd_pledged_real: float (nullable = true)
 |-- usd_goal_real: float (nullable = true)
 |-- unix_dateLaunched: long (nullable = true)
 |-- unix_dateDeadLine: long (nullable = true)
 |-- quarter: integer (nullable = true)
 |-- label: integer (nullable = true)



[Row(main_category=4, backers=0.0, country=7, usd_pledged_real=0.0, usd_goal_real=1533.949951171875, unix_dateLaunched=1439272800, unix_dateDeadLine=1444370400, quarter=3, label=0)]

Creamos el vector ensamblador

In [170]:
# Vector Ensamblador
df_assembler = VectorAssembler(inputCols=['main_category',
                                          'backers',
                                          'country',
                                          'usd_pledged_real',
                                          'usd_goal_real',
                                          'unix_dateLaunched',
                                          'unix_dateDeadLine',
                                          'quarter'], outputCol="features")
df_num = df_assembler.transform(df_num)

Visualizamos el vector ensamblado compuesto por features y label

In [171]:
# visulizacion de vector ensamblado compuesto por features y label
df_num.printSchema()
df_num.head(1)

root
 |-- main_category: integer (nullable = true)
 |-- backers: float (nullable = true)
 |-- country: integer (nullable = true)
 |-- usd_pledged_real: float (nullable = true)
 |-- usd_goal_real: float (nullable = true)
 |-- unix_dateLaunched: long (nullable = true)
 |-- unix_dateDeadLine: long (nullable = true)
 |-- quarter: integer (nullable = true)
 |-- label: integer (nullable = true)
 |-- features: vector (nullable = true)



[Row(main_category=4, backers=0.0, country=7, usd_pledged_real=0.0, usd_goal_real=1533.949951171875, unix_dateLaunched=1439272800, unix_dateDeadLine=1444370400, quarter=3, label=0, features=DenseVector([4.0, 0.0, 7.0, 0.0, 1533.95, 1439272800.0, 1444370400.0, 3.0]))]

Partimos a continuación el set de datos en 75% training y 25% testing:

In [172]:
# Particion del data set
model_df=df_num.select(["features","label"])
train,test = model_df.randomSplit([0.75,0.25])


print(f"Size of train Dataset : {train.count()}" )
print(f"Size of test Dataset : {test.count()}" )

Size of train Dataset : 279199
Size of test Dataset : 92867


Creamos el Regresor Lineal:

In [173]:
lr = LinearRegression()

Entrenamos el modelo de regresión lineal:

In [174]:
# Fit the model, le llamamos lr_model
lr_model = lr.fit(train)

Creamos el dataframe de prediciones (predictions_df) a partir del modelo de entrenamiento y el conjunto de datos test:

In [175]:
predictions_df = lr_model.transform(test)


Visualizamos el contenido de predictions_df:

In [176]:
# visulizacion de predictions_df
predictions_df.select("prediction","label","features").show(5)

+-------------------+-----+--------------------+
|         prediction|label|            features|
+-------------------+-----+--------------------+
|0.31994407403419567|    0|[1.0,0.0,1.0,0.0,...|
|0.32429831992600133|    0|[1.0,0.0,1.0,0.0,...|
|0.27814733054187335|    0|[1.0,0.0,1.0,0.0,...|
| 0.3011440874346858|    0|[1.0,0.0,1.0,0.0,...|
| 0.3167592488834744|    0|[1.0,0.0,1.0,0.0,...|
+-------------------+-----+--------------------+
only showing top 5 rows



Ahora, evaluamos el modelo de Regresión Lineal, con los datos de TEST:

In [177]:
# evaluacion del modelo, le llamaremos model_predictions
from pyspark.ml.evaluation import RegressionEvaluator
lr_evaluator = RegressionEvaluator(predictionCol="prediction", \
                 labelCol="label",metricName="r2")

Imprimimos el valor de R2:

In [178]:
# valor de R2
print("R Squared (R2) on test data = %g" % lr_evaluator.evaluate(predictions_df))

R Squared (R2) on test data = 0.0309135


Imprimimos el valor del meanSquaredError:

In [179]:
# valor del meanSquaredError
test_result = lr_model.evaluate(test)
print("Root Mean Squared Error (RMSE) on test data = %g" % test_result.rootMeanSquaredError)

Root Mean Squared Error (RMSE) on test data = 0.471917


# Regresión con Árboles de Decisión

Importamos la librería DecisionTreeRegressor:

In [180]:
# import lib
from pyspark.ml.regression import DecisionTreeRegressor

Creamos el Regresor DT, le llamaremos dec_tree:

In [181]:
# dec_tree
dt = DecisionTreeRegressor(featuresCol ='features', labelCol = 'label')

Entrenamos el modelo:

In [182]:
# Train model, le llamaremos dec_tree_model
dec_tree_model = dt.fit(train)

Desplegamos las featureImportances:

In [183]:
dec_tree_model.featureImportances

SparseVector(8, {1: 0.5001, 3: 0.2598, 4: 0.2401})

Evaluamos el modelo con los datos de entrenamiento:

In [184]:
# Make predictions, le llamaremos model_predictions 
model_predictions = dec_tree_model.transform(test)

In [185]:
# visualizamos

model_predictions.show()

+--------------------+-----+------------------+
|            features|label|        prediction|
+--------------------+-----+------------------+
|[1.0,0.0,1.0,0.0,...|    0|0.1319734578520521|
|[1.0,0.0,1.0,0.0,...|    0|0.1319734578520521|
|[1.0,0.0,1.0,0.0,...|    0|               0.0|
|[1.0,0.0,1.0,0.0,...|    0|               0.0|
|[1.0,0.0,1.0,0.0,...|    0|               0.0|
|[1.0,0.0,1.0,0.0,...|    0|               0.0|
|[1.0,0.0,1.0,0.0,...|    0|               0.0|
|[1.0,0.0,1.0,0.0,...|    0|               0.0|
|[1.0,0.0,1.0,0.0,...|    0|               0.0|
|[1.0,0.0,1.0,0.0,...|    0|               0.0|
|[1.0,0.0,1.0,0.0,...|    0|               0.0|
|[1.0,0.0,1.0,0.0,...|    0|               0.0|
|[1.0,0.0,1.0,0.0,...|    0|               0.0|
|[1.0,0.0,1.0,0.0,...|    0|               0.0|
|[1.0,0.0,2.0,0.0,...|    0|0.1319734578520521|
|[1.0,0.0,2.0,0.0,...|    0|               0.0|
|[1.0,0.0,2.0,0.0,...|    0|               0.0|
|[1.0,0.0,2.0,0.0,...|    0|            

Usando RegressionEvaluator calculamos e imprimimos el valor de las metricas R2 y RMSE:

In [186]:
# R2 value of the model on test data 
dt_evaluator = RegressionEvaluator(metricName='r2')
dt_r2 = dt_evaluator.evaluate(model_predictions)
print(f'The r-square value of DecisionTreeRegressor is {dt_r2}')

# RMSE value of the model on test data
dt_evaluator2 = RegressionEvaluator(metricName='rmse')
dt_rmse = dt_evaluator2.evaluate(model_predictions)
print(f'The rmse value of DecisionTreeRegressor is {dt_rmse}')

The r-square value of DecisionTreeRegressor is 0.8990387532522395
The rmse value of DecisionTreeRegressor is 0.15232182732278568


# Gradient-Boosted Tree Regressor

Importamos a GBTRegressor

In [187]:
# import
from pyspark.ml.regression import GBTRegressor

Creamos el Regresor GBTR, le llamaremos gbt:

In [188]:
# regresor
gbt = GBTRegressor(featuresCol = 'features', labelCol = 'label', maxIter=10)

Entrenamos el modelo:

In [189]:
# Train model, le llamaremos gbt_model
gbt_model = gbt.fit(train)

Desplegamos las featureImportances:

In [190]:
#Importances
gbt_model.featureImportances

SparseVector(8, {0: 0.0008, 1: 0.3753, 2: 0.0004, 3: 0.34, 4: 0.2826, 5: 0.0004, 6: 0.0004, 7: 0.0001})

Evaluamos el modelo con los datos de entrenamiento, le llamaremos model_predictions:

In [191]:
# Model
model_predictions = gbt_model.transform(test)

Desplegamos los valores del model_predictions

In [192]:
# show 
model_predictions.select('prediction', 'label', 'features').show(5)

+--------------------+-----+--------------------+
|          prediction|label|            features|
+--------------------+-----+--------------------+
|  0.0653057400078152|    0|[1.0,0.0,1.0,0.0,...|
| 0.06530574000781526|    0|[1.0,0.0,1.0,0.0,...|
|0.003432054010509...|    0|[1.0,0.0,1.0,0.0,...|
|-0.00100969365399...|    0|[1.0,0.0,1.0,0.0,...|
|-7.48298619196343E-4|    0|[1.0,0.0,1.0,0.0,...|
+--------------------+-----+--------------------+
only showing top 5 rows



Usando RegressionEvaluator calculamos e imprimimos el valor de las metricas R2 y RMSE:

In [193]:
#Select (prediction, true label) and compute test error
gbt_evaluator = RegressionEvaluator(
    labelCol="label", predictionCol="prediction", metricName="rmse")

gbt_evaluatorR2 = RegressionEvaluator(
    labelCol="label", predictionCol="prediction", metricName="r2")
# R2 value of the model on test data 
r2 = gbt_evaluatorR2.evaluate(model_predictions)
print("R2 on test data = %g" % r2)



# RMSE value of the model on test data 
rmse = gbt_evaluator.evaluate(model_predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

R2 on test data = 0.933419
Root Mean Squared Error (RMSE) on test data = 0.123697


## Conclusiones

|Algorithm|R2|RMSE|
|-----|:-------:|-----------
|Regresion Lineal|   0.03|0.47
|Decision Tree|   0.89|0.15
|Gradient-Boosted Tree|   0.93|0.12

La regresión lineal es aplicada a problemas como:

- Venta de un producto; precios, rendimiento y parámetros de riesgo
- Generar información sobre el comportamiento del consumidor, la rentabilidad y otros factores comerciales.
- Evaluación de tendencias; hacer estimaciones y pronósticos
- Determinar la efectividad del marketing, los precios y las promociones en las ventas de un producto.
- Realización de estudios de mercado y análisis de resultados de encuestas a clientes.

Cuando sabemos que la relación entre la variable independiente y la dependiente tiene una relación lineal, este algoritmo es el mejor para usar porque es el menos complejo en comparación con otros algoritmos que también intentan encontrar la relación entre la variable independiente y la dependiente.

Para este problema en concreto sobre tendencias de fracaso o exito de un proyecto el algoritmo con menor error es más apegado a una regresion lineal.

Por otro lado Gradient Boosted Tree, construye árboles uno a la vez, donde cada nuevo árbol ayuda a corregir los errores cometidos por un árbol previamente entrenado.

Cuando Gradient-Boosted Tree se realiza junto con la regresión lineal, no es más que otro modelo lineal sobre el modelo lineal existente. Esto puede entenderse intuitivamente como agregar algo a los coeficientes ya encontrados, y si la regresión lineal ya ha encontrado los mejores coeficientes, será inútil.

Hay dos ventajas de Gradient Boosted Tree con regresión lineal, primero poder regularizar los valores de los coeficientes y ayudar en el caso de overfit. Segundo, cuando los datos tienen alguna forma compleja no lineal. Los métodos de refuerzo lo ayudan a evolucionar lentamente con los datos.

Para este caso parece que la regresion lineal se ajusta bien, entonces cuando Gradient Boosted Tree se ajusta a los datos, luego el siguiente modelo se basa en los residuos del modelo anterior, pero los residuos de los modelos lineales no pueden ajustarse a otro modelo lineal y por eso al finalizar el algoritmo aumenta el error.


Decision Tree y Gradient-Boosted Tree son más utilizado cuando no hay una relacion lineal entre las variables, esto debido al ajuste que realizan conforme van profundizando en el arbol, por lo que al tener tan buen resultado con linear regresion podemos asumir que el modelo tiene un mejor fit con regresion lineal, entonces basicamente no estamos usando los algoritmos mas adecuados al problema por lo que el error es un poco mayor, lo cual es mas notorio con el resultado del error cudratico.

Sin embargo, los 3 algoritmos muestran un error RMSE es la raíz cuadrada de la varianza de los residuos. Indica el ajuste absoluto del modelo a los datos: qué tan cerca están los puntos de datos observados de los valores pronosticados del modelo. RMSE es una medida absoluta de ajuste, como la raíz cuadrada de una varianza, RMSE puede interpretarse como la desviación estándar de la varianza inexplicada, y tiene la propiedad útil de estar en las mismas unidades que la variable de respuesta. Los valores más bajos de RMSE indican un mejor ajuste. RMSE es una buena medida de la precisión con que el modelo predice la respuesta, y es el criterio más importante para el ajuste si el propósito principal del modelo es la predicción.
